In [1]:
from IPython.display import display
from flair.embeddings import BytePairEmbeddings, DocumentPoolEmbeddings
from src.ned import BasicFlairNED

import warnings
warnings.filterwarnings('ignore')

In [2]:
byte_pair_emb = BytePairEmbeddings('en', dim=300)
pool_doc_emb = DocumentPoolEmbeddings([byte_pair_emb], fine_tune_mode='nonlinear')
flair_ned = BasicFlairNED(pool_doc_emb)

In [3]:
examples = [
    ("London", "Crime is on the rise in London, which is of grave concern to the regional government in Ontario"),
    ("jaguar", "the prey saw the jaguar cross the jungle"),
    ("blackberry", "The blackberry is an edible fruit produced by many species in the Rubus genus in the Rosaceae family, hybrids among these species within the Rubus subgenus, and hybrids between the Rubus and Idaeobatus subgenera"),
    ("BlackBerry", "During the second financial quarter of 2013, BlackBerry sold 6.8 million handsets, but was eclipsed by the sales of competitor Nokia's Lumia model for the first time"),
    ("string", "You can manipulate strings in Python"),
    ("python", "A python is typically not venomous"),
    ("queen", "In England, the queen still has a lot of power"),
]

In [4]:
%time # 3.4s
for mention, context in examples:
	display(flair_ned.link(mention, context))

Wall time: 0 ns


,Entity,Description,Confidence
0,London Recordings,record label headquartered in the United Kingdom,0.757874
1,London,capital and largest city of the United Kingdom,0.741486
2,London,"city in Southwestern Ontario, Canada",0.741486
3,London,"city in Madison County, Ohio, United States",0.741486
4,London,"city in Kentucky, United States",0.741486


,Entity,Description,Confidence
0,Panthera onca,species of big cat native to the Americas,0.726609
1,SEPECAT Jaguar,attack aircraft family by SEPECAT,0.649750
2,Atari Jaguar,home video game console,0.619729
3,Jaguar Cars,former British car company,0.605421


,Entity,Description,Confidence
0,Morus nigra,species of plant,0.853793
1,Rubus subg. Rubus,subgenus of plants,0.853463
2,blackberry,fruit of Rubus subg. Rubus,0.842853
3,BlackBerry,line of wireless handheld devices and services,0.561625
4,BlackBerry,enterprise software and the Internet of things...,0.561625


,Entity,Description,Confidence
0,BlackBerry,line of wireless handheld devices and services,0.775363
1,BlackBerry,enterprise software and the Internet of things...,0.775363
2,blackberry,fruit of Rubus subg. Rubus,0.642766
3,Morus nigra,species of plant,0.544809
4,Rubus subg. Rubus,subgenus of plants,0.544297


,Entity,Description,Confidence
0,string,"sequence of characters, data type",0.558625
1,string,"musical instrument part, made from metal or pl...",0.558625
2,string,"long, flexible structure made from fibers twis...",0.558625
3,string,hypothetical physical entity,0.558625


,Entity,Description,Confidence
0,CPython,Python reference implementation,0.786023
1,Python,general-purpose programming language,0.620562
2,Python,genus of reptiles,0.620562
3,Python,earth-dragon of Delphi,0.620562


,Entity,Description,Confidence
0,Elizabeth II,"queen of the UK, Canada, Australia, and New Ze...",0.800449
1,queen regnant,female monarch who rules a country in her own ...,0.791392
2,monarch,person at the head of a monarchy,0.774132
3,drag queen,male person who dresses and acts with exaggera...,0.722036
4,Queen,British rock band (1970-),0.677904
